In [62]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Rickthom"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#animal_types = df['animal_type'].dropna().unique()
#animal_types = [animal for animal in animal_types if animal]

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = '/home/richardseabri_snhu/Desktop/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    # Image display with unique identifier
    html.Center(
        html.Div([
            html.A(
                html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px'}),
                href='https://www.snhu.edu',  # URL to the client’s homepage
                target='_blank'  # Opens the link in a new tab
            ),
            html.Br(),
            html.B(html.H1('CS-340 Dashboard - Richard Seabridge'))  # Unique identifier with your name
        ])
    ),
    html.Hr(),

    # Interactive filtering options
html.Div([
        html.H4('Rescue Type', style={'text-align': 'center', 'color': 'purple'}),
        dcc.RadioItems(
            id='filter-options',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',  # Default option
            labelStyle={'display': 'inline-block'}
        )
    ], style={'text-align': 'left', 'margin-bottom': '20px'}),
    
    html.Hr(),

    # Data Table
 dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Display 10 rows at a time
        row_selectable='single',  # Enable single row selection
        sort_action='native',  # Allow sorting of columns
        selected_rows=[0],  # Set the first row as selected by default
        style_table={'overflowX': 'auto'},  # Responsive table
        style_data={'whiteSpace': 'normal', 'height': 'auto'},  # Auto height for rows
    ),
    html.Br(),
    html.Hr(),
html.Div(className='row', style={'display': 'flex'}, children=[
        # Graph with ID
        html.Div(id='graph-id', className='col s12 m6'),  # Ensure graph-id exists

        # Map with ID
        html.Div(id='map-id', className='col s12 m6')
    ])
])
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
    
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-options', 'value')])  # Changed to 'filter-options'
def update_dashboard(filter_type):
    # Define the filtering criteria based on the provided research table
    rescue_criteria = {
        'Water Rescue': df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']),
        'Mountain Rescue': df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']),
        'Disaster Rescue': df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])
    }
    
    if filter_type == 'Reset':
        filtered_df = df
    else:
        filtered_df = df[rescue_criteria[filter_type]]

    return filtered_df.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return None  # Return None if there is no data

    dff = pd.DataFrame.from_dict(viewData)

    if not dff.empty:
        breed_counts = dff['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        
        threshold = 0.02 * breed_counts['count'].sum()  # Example threshold for grouping smaller breeds
        breed_counts['breed'] = np.where(breed_counts['count'] < threshold, 'Other', breed_counts['breed'])

        breed_counts = breed_counts.groupby('breed').sum().reset_index()

        figure = px.pie(breed_counts, names='breed', values='count', title='Animal Breeds Distribution', hole=0.4)
        figure.update_traces(textposition='outside', textinfo='percent+label')

        return [dcc.Graph(figure=figure)]
    else:
        return [html.Div("No data available to display.")]
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'data')]  # Adjust the input if necessary
)
def update_table_style(data):
    if data is None or len(data) == 0:
        return []  # Return an empty list if no data is available

    # Example condition for styling
    return [
        {
            'if': {
                'filter_query': '{column_name} > some_value',
                'column_id': 'column_name'
            },
            'backgroundColor': '#FFDDDD'  # Example style
        },
        # Add more conditions as needed
    ]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    
    if selected_rows is None or len(selected_rows) == 0:
        row = 0  # Default to the first row if none selected
    else:
        row = selected_rows[0]  # Use the first selected row index

    if row >= len(dff):
        return []

    latitude = dff.iloc[row, 13]  # Adjust index based on your DataFrame
    longitude = dff.iloc[row, 14]  # Adjust index based on your DataFrame

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[latitude, longitude],
                              children=[
                                  dl.Tooltip(dff.iloc[row, 4]),  # Assuming this is the breed
                                  dl.Popup([
                                      html.H1("Animal Name"),
                                      html.P(dff.iloc[row, 9])  # Assuming this is the name
                                  ])
                              ])
               ])
    ]



if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:13286/
